# Text Processing

There are a number of generalized steps in processing various text values.


## Unicode Normalization

Unicode encoding supports a number of "normal forms" and the concept of ["Unicode equivalence"](https://en.wikipedia.org/wiki/Unicode_equivalence). In order to make simple comparisons between strings possible, and efficient, all strings involved should be [normalized](https://en.wikipedia.org/wiki/Text_normalization) to a known standard, either combined (utilizing characters / code points that combine accents with the letter they modify) or utilizing [combining characters](https://en.wikipedia.org/wiki/Combining_character).

Additionally, there is an excellent library called [FTFY](https://ftfy.readthedocs.org/) (it _Fixes Text For You_) that can resolve incorrect decoding referred to as [mojibake](https://en.wikipedia.org/wiki/Mojibake), and other issues.

In [48]:
from typing import List, Union
from unicodedata import combining, normalize

In [49]:
def uninorm(value:Union[bytes,str], encoding:str='utf8', fallback:str='Windows-1252') -> str:
	if isinstance(value, bytes):
		try:
			value = value.decode(encoding)
		except UnicodeDecodeError:
			value = value.decode(fallback)
	
    # value = fix_text(value)  # If utilizing the excellent FTFY library.
	value = normalize('NFC', value)  # Combined form.
	value = value.replace('\r\n', '\n')  # Also eliminate extraneous Windows end-of-line markers.
	
	return value

## Eliminating Combining Characters

Sometimes you just want ASCII out the other side. However, most "accented" characters can be represented without their accent without too much loss of fidelity; wetware is quite good at correcting minor deficciencies like that. Luckily, by decomposing into a combining character form we can very easily run through the resulting characters and throw away those identified as combining.

This has the added benefit of eliminating most "[Zalgo text](https://en.wikipedia.org/wiki/Combining_character#Zalgo_text)", a disruptive abuse of stackable combining characters to force rendering to escape a container and obscure surrounding text.

In [50]:
def noncombining(value:str) -> str:
	return "".join(c for c in normalize('NFKD', value) if not combining(c))

In [51]:
noncombining("Alice Zoë Bevan-McGregor")

'Alice Zoe Bevan-McGregor'

## URL-Safe "Slugification"

An extremely common pattern is that of "slugification", that is, transforming a textual label into a value suitable for use within a URI, e.g. as a path element. Typically used to support "SEO-friendly URLs" where the address can be memorable and not just an abstract identifier.

In [52]:
from re import compile as re, Pattern

In [53]:
def urlsafe(name:str, collection:List[str]=[], replacement:str='-', unsafe:Pattern=re(r'\W+')) -> str:
    base = unsafe.sub(replacement, uninorm(name)).strip(replacement)
    suffix = 0
    
    while True:
        if (result := f"{base}{replacement if suffix else ''}{suffix if suffix else ''}") not in collection:
            break
        suffix += 1
    
    return result

In [54]:
urlsafe("Alice Zoë Bevan-McGregor")

'Alice-Zoë-Bevan-McGregor'

In [55]:
# If you have a collection of values, to ensure collisions are avoided, pass it in.
urlsafe("Words", ["Words", "Words-1"])

'Words-2'